In [53]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [54]:
def date_time(table_cells):

    return [date_time.strip() for date_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):

    out = ''.join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2==0])[0:-1]

    return out

def landing_status(table_cells):

    out = [i for i in table_cells.strings][0]

    return out

def get_mass(table_cells):

    mass = unicodedata.normalize("NFKD", table_cells.text).strip()

    if mass:

        mass.find("kg")
        new_mass = mass[0:mass.find("kg")+2]

    else:

        new_mass = 0

    return new_mass

def extract_column_from_header(row):

    if (row.br):

        row.br.extract()

    if row.a:

        row.a.extract()

    column_name = ' '.join([str(item) for item in row.contents])

    if not(column_name.strip().isdigit()):

        return column_name






In [55]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [56]:
url = static_url
response = requests.get(url)

if response.status_code == 200:
  print("Request Successful!")

else:
  print("Request Failed with status code!", response.status_code)

Request Successful!


In [57]:
soup = BeautifulSoup(response.text, "html.parser")

In [58]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [59]:
html_tables = soup.find_all('table')

In [60]:
first_table = html_tables[2]
print(first_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [61]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
headers = first_table.find_all('th')
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
for i in headers:
  column_name = extract_column_from_header(i)
  if column_name is not None and len(column_name) > 0:
    column_names.append(column_name)
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
print(column_names)

['Flight No.\n', 'Date and time ( )\n', '  <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup> \n', 'Launch site\n', 'Payload <sup class="reference" id="cite_ref-Dragon_12-0"></sup> \n', 'Payload mass\n', 'Orbit\n', 'Customer\n', 'Launch outcome\n', '\n']


In [62]:
for i in headers:
  print(i.text)

Flight No.

Date andtime ()

 [b]

Launch site

Payload

Payload mass

Orbit

Customer

Launchoutcome



1

2

3

4

5

6

7



In [64]:
launch_dict = dict.fromkeys(column_names)

del launch_dict['Date and time ( )\n']
del launch_dict['  <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup> \n']
del launch_dict['\n']


launch_dict['Flight No.'] = launch_dict.pop('Flight No.\n')
launch_dict['Launch site'] = launch_dict.pop('Launch site\n')
launch_dict['Payload'] = launch_dict.pop('Payload <sup class="reference" id="cite_ref-Dragon_12-0"></sup> \n')
launch_dict['Payload mass'] = launch_dict.pop('Payload mass\n')
launch_dict['Orbit'] = launch_dict.pop('Orbit\n')
launch_dict['Customer'] = launch_dict.pop('Customer\n')
launch_dict['Launch outcome'] = launch_dict.pop('Launch outcome\n')

In [65]:
print(launch_dict)

{'Flight No.': None, 'Launch site': None, 'Payload': None, 'Payload mass': None, 'Orbit': None, 'Customer': None, 'Launch outcome': None}


In [66]:
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []

launch_dict['Version Booster'] = []
launch_dict['Booser landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [70]:
extracted_row = 0

for table_number, table in enumerate(html_tables):
    for rows in table.find_all("tr"):
      if rows.th:
          if rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()

      else:
        flag = False
      rows=rows.find_all('td')
      if flag:
        extracted_row += 1
        launch_dict['Flight No.'].append(flight_number)
        print(flight_number)

        datetimelist = date_time(rows[0])
        date = datetimelist[0].strip(',')
        launch_dict['Date'].append(date)
        print(date)

        launch_dict['Time'].append(time)
        time = datetimelist[1]
        print(time)

        launch_dict['Version Booster'].append(bv)
        bv = booster_version(rows[1])
        if not(bv):
          bv = rows[1].a.string
        print(bv)

        launch_site = rows[2].a.string
        launch_dict['Launch site'].append(launch_site)

        print(launch_site)

        payload = rows[3].a.string
        launch_dict['Payload'].append(payload)
        print(payload)

        payload_mass = get_mass(rows[4])
        launch_dict['Payload mass'].append(payload_mass)
        print(payload_mass)

        orbit = rows[5].a.string
        launch_dict['Orbit'].append(orbit)
        print(orbit)

        customer = rows[6].a.string
        launch_dict['Customer'].append(customer)
        print(customer)

        launch_outcome = list(rows[7].strings)[0]
        launch_dict['Launch outcome'].append(launch_outcome)
        print(launch_outcome)

        booster_landing = landing_status(rows[8])
        launch_dict['Booser landing'].append(booster_landing)
        print(booster_landing)

1
4 June 2010
18:45
F9 v1.0B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
2
8 December 2010
15:43
F9 v1.0B0004.1
CCAFS
Dragon
3
22 May 2012
07:44
F9 v1.0B0005.1
CCAFS
Dragon
4
8 October 2012
00:35
F9 v1.0B0006.1
CCAFS
SpaceX CRS-1
5
1 March 2013
15:10
F9 v1.0B0007.1
CCAFS
SpaceX CRS-2
6
29 September 2013
16:00
F9 v1.1B1003
VAFB
CASSIOPE
7
3 December 2013
22:41
F9 v1.
CCAFS
SES-8
8
6 January 2014
22:06
F9 v1.
CCAFS
Thaicom 6
9
18 April 2014
19:25
F9 v1.
Cape Canaveral
SpaceX CRS-3
10
14 July 2014
15:15
F9 v1.
Cape Canaveral
Orbcomm-OG2
11
5 August 2014
08:00
F9 v1.
Cape Canaveral
AsiaSat 8
12
7 September 2014
05:00
F9 v1.1[8
Cape Canaveral
AsiaSat 6
13
21 September 2014
05:52
F9 v1.1[8
Cape Canaveral
SpaceX CRS-4
14
10 January 2015
09:47
F9 v1.1[8
Cape Canaveral
SpaceX CRS-5
15
11 February 2015
23:03
F9 v1.1[8
Cape Canaveral
DSCOVR
16
2 March 2015
03:50
F9 v1.1[8
Cape Canaveral
ABS-3A
17
14 April 2015
20:10
F9 v1.1[8
Cape Canaveral
SpaceX CRS-6
18
27 April 2015
23:03
F9 v1.1[8
Cape 

In [72]:
df = pd.DataFrame({key: pd.Series(value) for key, value in launch_dict.items()})

In [73]:
df.to_csv('spacex_web_scraped.csv', index = False)